In [ ]:
# ==============================
# Customer Support Ticket Classification
# ==============================

# Install dependencies if needed
# !pip install transformers datasets evaluate accelerate scikit-learn


import pandas as pd
from datasets import Dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score

# ==============================
# 1. Load Dataset
# ==============================
# Load your dataset (adjust filename as needed)
df = pd.read_csv("/content/customer_support_tickets.csv")

# Keep only relevant columns
df = df[["Ticket Description", "Ticket Type"]].dropna()

# Rename for consistency
df = df.rename(columns={"Ticket Description": "text", "Ticket Type": "label"})

# Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Train/test split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Encode labels
labels = list(df["label"].unique())
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for i, l in enumerate(labels)}

train_dataset = train_dataset.map(lambda x: {"label": label2id[x["label"]]})
test_dataset = test_dataset.map(lambda x: {"label": label2id[x["label"]]})


# ==============================
# 2. Zero-Shot Classification
# ==============================
zero_shot_clf = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

sample_text = df["text"].iloc[0]
print("\n--- Zero-Shot Example ---")
print("Text:", sample_text)
result = zero_shot_clf(sample_text, labels)
print("Predicted:", result["labels"][0])


# ==============================
# 3. Few-Shot Prompt Example
# ==============================
few_shot_prompt = f"""
Classify the following support ticket into one of these categories: {labels}.

Examples:
Ticket: "I cannot log into my account"
Category: Account

Ticket: "I was double charged for my purchase"
Category: Billing

Now classify this ticket:
Ticket: "{df['text'].iloc[1]}"
Category:
"""

few_shot_clf = pipeline("text-generation", model="gpt2")
output = few_shot_clf(few_shot_prompt, max_new_tokens=20)
print("\n--- Few-Shot Example ---")
print("Prediction:", output[0]["generated_text"])


# ==============================
# 4. Fine-Tuning DistilBERT
# ==============================
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_enc = train_dataset.map(tokenize_fn, batched=True)
test_enc = test_dataset.map(tokenize_fn, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels_true = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels_true)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("\n--- Fine-tuning model ---")
trainer.train()


# ==============================
# 5. Evaluate & Compare
# ==============================
print("\n--- Evaluating Fine-tuned Model ---")
fine_tuned_results = trainer.evaluate()
print("Fine-tuned Accuracy:", fine_tuned_results["eval_accuracy"])

# Zero-shot evaluation on test set
true_labels = [id2label[l] for l in test_dataset["label"]]
pred_labels = []
for t in test_dataset["text"]:
    res = zero_shot_clf(t, labels)
    pred_labels.append(res["labels"][0])

zero_shot_acc = accuracy_score(true_labels, pred_labels)

print("\n--- Performance Comparison ---")
print(f"Zero-Shot Accuracy: {zero_shot_acc:.4f}")
print(f"Fine-Tuned Accuracy: {fine_tuned_results['eval_accuracy']:.4f}")


Map:   0%|          | 0/338 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Device set to use cpu



--- Zero-Shot Example ---
Text: I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.
Predicted: Product inquiry


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Few-Shot Example ---
Prediction: 
Classify the following support ticket into one of these categories: ['Technical issue', 'Billing inquiry', 'Cancellation request', 'Product inquiry', 'Refund request'].

Examples:
Ticket: "I cannot log into my account"
Category: Account

Ticket: "I was double charged for my purchase"
Category: Billing

Now classify this ticket:
Ticket: "I'm having an issue with the {product_purchased}. Please assist.

If you need to change an existing product.

I'm having an issue with the {product_purchased}. Please assist.

If The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly."
Category:

Ticket: 'I'm having an issue with the {product_purchased}. Please


Map:   0%|          | 0/338 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 27.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [ ]:
# ==============================
# Customer Support Ticket Classification
# ==============================

# Install dependencies if needed
# !pip install -U transformers datasets evaluate scikit-learn accelerate

import pandas as pd
from datasets import Dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score

# ==============================
# 1. Load Dataset
# ==============================
df = pd.read_csv("/content/customer_support_tickets.csv")

# Keep only relevant columns
df = df[["Ticket Description", "Ticket Type"]].dropna()

# Rename for consistency
df = df.rename(columns={"Ticket Description": "text", "Ticket Type": "label"})

# Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Train/test split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Encode labels
labels = list(df["label"].unique())
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for i, l in enumerate(labels)}

train_dataset = train_dataset.map(lambda x: {"label": label2id[x["label"]]})
test_dataset = test_dataset.map(lambda x: {"label": label2id[x["label"]]})


# ==============================
# 2. Zero-Shot Classification
# ==============================
zero_shot_clf = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

sample_text = df["text"].iloc[0]
print("\n--- Zero-Shot Example ---")
print("Text:", sample_text)
result = zero_shot_clf(sample_text, labels)
print("Predicted:", result["labels"][0])


# ==============================
# 3. Few-Shot Prompt Example
# ==============================
few_shot_prompt = f"""
Classify the following support ticket into one of these categories: {labels}.

Examples:
Ticket: "I cannot log into my account"
Category: Account

Ticket: "I was double charged for my purchase"
Category: Billing

Now classify this ticket:
Ticket: "{df['text'].iloc[1]}"
Category:
"""

few_shot_clf = pipeline("text-generation", model="gpt2")
output = few_shot_clf(few_shot_prompt, max_new_tokens=20)
print("\n--- Few-Shot Example ---")
print("Prediction:", output[0]["generated_text"])


# ==============================
# 4. Fine-Tuning DistilBERT
# ==============================
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_enc = train_dataset.map(tokenize_fn, batched=True)
test_enc = test_dataset.map(tokenize_fn, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels_true = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels_true)

# ⚠️ Minimal TrainingArguments for very old Transformers
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to=[]   # 🚀 disables wandb logging

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("\n--- Fine-tuning model ---")
trainer.train()


# ==============================
# 5. Evaluate & Compare
# ==============================
print("\n--- Evaluating Fine-tuned Model ---")
fine_tuned_results = trainer.evaluate()
print("Fine-tuned Accuracy:", fine_tuned_results["eval_accuracy"])

# Zero-shot evaluation on test set
true_labels = [id2label[l] for l in test_dataset["label"]]
pred_labels = []
for t in test_dataset["text"]:
    res = zero_shot_clf(t, labels)
    pred_labels.append(res["labels"][0])

zero_shot_acc = accuracy_score(true_labels, pred_labels)

print("\n--- Performance Comparison ---")
print(f"Zero-Shot Accuracy: {zero_shot_acc:.4f}")
print(f"Fine-Tuned Accuracy: {fine_tuned_results['eval_accuracy']:.4f}")


Map:   0%|          | 0/338 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Device set to use cpu



--- Zero-Shot Example ---
Text: I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.
Predicted: Product inquiry


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss



--- Evaluating Fine-tuned Model ---


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Fine-tuned Accuracy: 0.18823529411764706

--- Performance Comparison ---
Zero-Shot Accuracy: 0.1647
Fine-Tuned Accuracy: 0.1882
